In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from tensorflow import keras 

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
# import keras
from tensorflow import keras

BASE_PATH = "models/"

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [13]:
import tensorflow_model_optimization as tfmot
import tempfile
def prune_model(model, x_train, y_train):
#     model = tf.keras.models.load_model(path_to_model)
    
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    
    # Prune param
    # Compute end step to finish pruning after 2 epochs.
    batch_size = 128
    epochs = 2
    end_step = np.ceil(x_train.shape[0] / batch_size).astype(np.int32) * epochs

    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                                   final_sparsity=0.20,                                                 
                                                                   begin_step=0,
                                                                   end_step=end_step)
    }
    
    # Define model for pruning.
    model_for_pruning = prune_low_magnitude(model, **pruning_params)
    
    # Prune require a compile
    model_for_pruning.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    logdir = tempfile.mkdtemp()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    model_for_pruning.fit(x_train, y_train,
                      batch_size=batch_size, epochs=epochs, callbacks=callbacks)
    
    # Get pruned model
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

    return model_for_export

In [14]:
import logging, os
logging.disable(logging.WARNING)
tf.autograph.set_verbosity(0)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [18]:
vgg16 = tf.keras.models.load_model('vgg_model/vgg16_cifar10_ver2.h5')

In [19]:
vgg16.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 2.3566 - accuracy: 0.7669


[2.356550693511963, 0.7669000029563904]

In [20]:
vgg16_pruned = prune_model(vgg16, x_train, y_train)

C:\Users\Zlx-nGen\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2
391/391 [==============================] - 27s 59ms/step - loss: 6.3441 - accuracy: 0.2182
Epoch 2/2
391/391 [==============================] - 19s 48ms/step - loss: 1.4102 - accuracy: 0.4601


In [21]:
sgd = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)
    
    # Prune require a compile
vgg16_pruned.compile(optimizer=sgd,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [22]:
vgg16_pruned.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1974 - accuracy: 0.5776


[1.203965187072754, 0.5773000121116638]

In [5]:
with tf.device('/cpu:0'):
    vgg16 = tf.keras.models.load_model('vgg_model/vgg16_cifar10.h5')
    vgg16_pruned = prune_model(vgg16, x_train, y_train)

C:\Users\Zlx-nGen\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2
 66/391 [====>.........................] - ETA: 8:18 - loss: 4.9649 - accuracy: 0.1940

KeyboardInterrupt: 

In [ ]:
vgg16 = tf.keras.models.load_model('vgg_model/vgg16_cifar10.h5')
# vgg16_pruned = prune_model(vgg16, x_train, y_train)
# vgg16_pruned.save('vgg_model_pruned/vgg16_cifar10.h5')

In [5]:
with tf.device('/cpu:0'):
    vgg16 = tf.keras.applications.VGG16()
    vgg19 = tf.keras.applications.VGG19()